In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 598 ms (started: 2021-08-21 00:32:41 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 ch16 #1  Aug 21, 2021'

git push origin master

origin	git@github.com:ustchope/eat_tensorflow2_in_30_days.git (fetch)
origin	git@github.com:ustchope/eat_tensorflow2_in_30_days.git (push)
[master bba3300] 更新 ch16 #1  Aug 20, 2021
 2 files changed, 151 insertions(+), 30 deletions(-)
 create mode 100644 ch16.ipynb


To git@github.com:ustchope/eat_tensorflow2_in_30_days.git
   8ff74c3..bba3300  master -> master


time: 12.8 s (started: 2021-08-20 21:03:52 +08:00)


In [2]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.98 s (started: 2021-08-21 00:32:48 +08:00)


In [ ]:
#初始化 Neptune 并创建新的运行

import neptune.new as neptune

run = neptune.init(project='ustchope/tf-keras-integration')

In [ ]:
# 导入并实例化 NeptuneCallback 并将其传递给 model.fit() 的回调参数。
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')

# model.fit(x_train, y_train,
#           epochs=5,
#           batch_size=64,
#           callbacks=[neptune_cbk])


# 记录超参数，按照真实数据填入即可

PARAMS = {'lr':0.005, 
          'momentum':0.9, 
          'epochs':10,
          'batch_size':32}

# log hyper-parameters
run['hyper-parameters'] = PARAMS

# 使用 RNN 和注意力的自然语言处理

当艾伦图灵在 1950 年想象他著名的图灵测试时，他的目标是评估机器匹配人类智能的能力。他本可以测试很多东西，例如识别图片中的猫、下棋、作曲或逃离迷宫的能力，但有趣的是，他选择了一项语言任务。更具体地说，他设计了一个聊天机器人，能够欺骗对话者认为它是人类。这个测试确实有它的弱点：一组硬编码规则可以愚弄毫无戒心或天真的人类（例如，机器可以针对某些关键字给出模糊的预定义答案；它可以假装在开玩笑或喝醉了，以通过其最奇怪的答案；或者它可以通过用自己的问题回答难题来逃避难题），并且人类智能的许多方面都被完全忽略了（例如，解释面部表情等非语言交流的能力，或学习手动任务的能力）。但该测试确实强调了一个事实，即掌握语言可以说是智人最大的认知能力。我们可以建造一台可以读写自然语言的机器吗？

自然语言任务的常见方法是使用循环神经网络。因此，我们将继续探索 RNN（在第 15 章中介绍），从字符 RNN 开始，经过训练以预测句子中的下一个字符。这将允许我们生成一些原始文本，在这个过程中我们将看到如何在一个很长的序列上构建一个 TensorFlow 数据集。我们将首先使用一个无状态的 RNN（它在每次迭代中学习文本的随机部分，没有关于文本其余部分的任何信息），然后我们将构建一个有状态的 RNN（它保留训练迭代之间的隐藏状态并继续阅读哪里它停止了，允许它学习更长的模式）。接下来，我们将构建一个 RNN 来执行情感分析（例如，阅读电影评论并提取评分者对电影的感觉），这次将句子视为单词序列，而不是字符。然后我们将展示如何使用 RNN 来构建能够执行神经机器翻译 (NMT) 的编码器-解码器架构。为此，我们将使用 TensorFlow Addons 项目提供的 seq2seq API。

在本章的第二部分，我们将研究注意力机制。 顾名思义，这些是神经网络组件，它们学会选择模型的其余部分在每个时间步应该关注的输入部分。 首先，我们将看到如何使用注意力来提高基于 RNN 的编码器-解码器架构的性能，然后我们将完全放弃 RNN，看看一个非常成功的仅注意力架构，称为 Transformer。 最后，我们将看看 2018 年和 2019 年 NLP 的一些最重要的进步，包括非常强大的语言模型，例如 GPT-2 和 BERT，它们都基于 Transformers。

让我们从一个简单而有趣的模型开始，它可以像莎士比亚一样写作（嗯，有点）。

## 使用字符 RNN 生成莎士比亚文本

在 2015 年一篇名为“循环神经网络的不合理有效性”的著名博客文章中，Andrej Karpathy 展示了如何训练 RNN 来预测句子中的下一个字符。 然后可以使用这个 Char-RNN 生成新颖的文本，一次一个字符。 这是 Char-RNN 模型在对莎士比亚的所有作品进行训练后生成的一小部分文本示例：
```
PANDARUS:
Alas, I think he shall be come approached and the day
When little srain would be attain’d into being never fed,
And who is but a chain and subjects of his death,
I should not sleep.
```

不完全是杰作，但令人印象深刻的是，该模型仅通过学习预测句子中的下一个字符就能够学习单词、语法、正确的标点符号等。 让我们从创建数据集开始，一步一步地看看如何构建 Char-RNN。

### 创建训练数据集

首先，让我们下载莎士比亚的所有作品，使用 Keras 方便的 get_file() 函数并从 Andrej Karpathy 的 Char-RNN 项目下载数据：

In [ ]:
shakespeare_url = "https://homl.info/shakespeare" # shortcut URL

filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

In [6]:
pwd

'/public/huangwei/node34/tutorial/eat_tensorflow2_in_30_days'

time: 2.98 ms (started: 2021-08-21 03:32:15 +08:00)
